In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import torch
import random
%pip install --index-url http://74.82.29.209:8090 nebula3_database==0.2.2 --trusted-host 74.82.29.209

In [ ]:
gr = []
from database.arangodb import DatabaseConnector
gdb = DatabaseConnector()
db = gdb.connect_db('prodemo')
query = 'FOR doc IN  s4_llm_output RETURN doc'
cursor = db.aql.execute(query)
for line in cursor:
    gr.append(line['candidate'])

In [ ]:
print(gr)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-2.7b", cache_dir="opt27_model/", torch_dtype=torch.float16).cuda()
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-2.7b", cache_dir="opt27_model/", use_fast=False)

In [ ]:

for para in gr:
    for event in para.split('.'):
        if event != '':
            set_seed(64)
            prompt = '''
            The woman is standing in front of a car window => people
            To the left of the bed is a table with a white vase on it => objects
            The room is dark and the only light is coming from a lamp on a table next to the couch => objects
            Man and woman are sitting at a dinner table with a man in a suit and tie => people
            The room is dimly lit with colored lights => objects
            She is looking at the camera and smiling => people
            The bush of pink flowers is in full bloom => objects
            {} => '''.format(event) 
            set_seed(64)
            input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
            generated_ids = model.generate(input_ids, do_sample=True, max_length=250)
            type_of_image = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            print("Event: ", event)
            #print(type_of_image[0])
            print(type_of_image[0].split(event)[1].split('\n')[0])

In [ ]:

event_ = ["a person plays a video game centered around ice age the movie","men are playing sports",
"girl with wings dances and sings the ole mcdonald song","person is looking down at a paper",
"has hands in pockets", "person is taking a tiny spoon and scooping up a heap of caviar",
"person is in the background conversing with others in the room",
"person is full of wounds and is standing beside his taxi in the middle of a \
street and giving instruction between the trucks that are parked on the street",
"person is standing on the table spraying a hose","person is crossing the busy street and watching a taxi speed off",
"person is sitting on a hospital bed and he is afraid of",
"person x is meanly looking at person y like he is about to throw a punch",
"person x is in uniform signaling for the life boat to lower and looking at person y in disgust",
"person sits in the lifeboat with his eyes closed looking ashamed and relieved",
"doctor examining person on a table",
"person x and person y embrace one another",
"at a fancy party pesron is trying to talk to the pretty woman in front of him"]

events_and_intents = []
for event in event_:
    set_seed(64)
    prompt = '''generate intent of action:
    person is facing the crowd while holding a microphone because he wants => to make a public statement.
    pesron is sitting at the bar in a sexy dress and waiting for someone because she wants => to flirt with a man.
    person is an officer that has handcuffed a citizen to himself because he wants => to arrest the citizen.
    person is carrying her meal to her table because she wants => to sit down to eat.
    {} because he or she wants => '''.format(event)
    print("EVENT: ",event)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    intents = []
    for i in range(5):
        
        generated_ids = model.generate(input_ids, do_sample=True, max_length=160)
        intent = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        #print(intent)
        intent = intent[0].split(event)[1].replace("\n",".").split(".")[0].split("=>")
        if len(intent) > 1:
            intent = intent[1]
        else:
            continue
        intents.append(intent.lstrip())
        print("    with the intent", intent.lstrip())
    events_and_intents.append((event, intents))
print(events_and_intents)

In [ ]:

event_ = ["a person plays a video game centered around ice age the movie","men are playing sports","girl with wings dances and sings the ole mcdonald song","person is looking down at a paper", "has hands in pockets", "person is taking a tiny spoon and scooping up a heap of caviar",
"person is in the background conversing with others in the room", \
"person is full of wounds and is standing beside his taxi in the middle of a \
street and giving instruction between the trucks that are parked on the street",
"person is standing on the table spraying a hose","person is crossing the busy street and watching a taxi speed off",\
"person is sitting on a hospital bed and he is afraid of", "person x is meanly looking at person y like he is about to throw a punch",
"person x is in uniform signaling for the life boat to lower and looking at person y in disgust",
"person sits in the lifeboat with his eyes closed looking ashamed and relieved",
"doctor examining person on a table",
"person x and person y embrace one another",
"at a fancy party pesron is trying to talk to the pretty woman in front of him"]

events_and_intents = []
for event in event_:
    set_seed(64)
    prompt = '''generate intent of action:
    person is facing the crowd while holding a microphone in order to => make a public statement.
    pesron is sitting at the bar in a sexy dress and waiting for someone in order to => flirt with a man.
    person is an officer that has handcuffed a citizen to himself in order to => arrest the citizen.
    person is carrying her meal to her table in order to => sit down to eat.
    {} in order to => '''.format(event)
    print("EVENT: ",event)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    intents = []
    for i in range(10):
        
        generated_ids = model.generate(input_ids, do_sample=True, max_length=160)
        intent = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        #print("1    --->",intent)
        intent = intent[0].split(event)[1].replace("\n",".").split(".")[0].split("=>")
        
        if len(intent) > 1:
            intent = intent[1]
        else:
            continue
        intents.append(intent.lstrip())
        print("2    --->",intent.lstrip())
    events_and_intents.append((event, intents))
print(events_and_intents)

In [ ]:
def get_prompy(query):
    #random.randrange(len(gr)-1)
    ev1 = gr[random.randrange(len(gr)-1)]
    ev2 = gr[random.randrange(len(gr)-1)]
    ev3 = gr[random.randrange(len(gr)-1)]
    ev4 = gr[random.randrange(len(gr)-1)]
    ev5 = gr[random.randrange(len(gr)-1)]
    ev6 = gr[random.randrange(len(gr)-1)]
   
    fsi=[ev1[0],ev1[1],ev2[0],ev2[1],ev3[0],ev3[1],ev4[0],ev4[1],ev5[0],ev5[1],ev6[0],ev6[1], query]
    prompt = '''generate intent of action:
        {} because he/she wants => {}.
        {} because he/she wants => {}.
        {} because he/she wants => {}.
        {} because he/she wants => {}.
        {} because he/she wants => {}.
        {} because he/she wants => {}.
        {} because he/she wants => '''.format(*fsi)
    return(prompt)

In [ ]:
events_and_intents = []
for event_ in gr:
    for sentence in event_.split("."):
        if sentence != "":
            event = sentence
            prompt = '''generate intent of action:
        person is facing the crowd while holding a microphone in order to => make a public statement.
        pesron is sitting at the bar in a sexy dress and waiting for someone in order to => flirt with a man.
        person is an officer that has handcuffed a citizen to himself in order to => arrest the citizen.
        person is carrying her meal to her table in order to => sit down to eat.
        {} in order to => '''.format(event)
            set_seed(64)
            print("EVENT: ",event)
            
            #print(prompt)
            input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
            intents = []
            for i in range(10):
                
                generated_ids = model.generate(input_ids, do_sample=True, max_length=300)
                intent = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
                #print("1    --->",intent)
                intent = intent[0].split(event)[1].replace("\n",".").split(".")[0].split("=>")
                
                if len(intent) > 1:
                    intent = intent[1]
                else:
                    continue
                intents.append(intent.lstrip())
                print("2    --->",intent.lstrip())
            events_and_intents.append({"db_id":event_[3],"movie":event_[2], "caption":event, "intention":intents})
        #print(events_and_intents)
        #input()

In [ ]:

event_ = ["a person plays a video game centered around ice age the movie","men are playing sports","girl with wings dances and sings the ole mcdonald song","person is looking down at a paper", "has hands in pockets", "person is taking a tiny spoon and scooping up a heap of caviar",
"person is in the background conversing with others in the room", \
"person is full of wounds and is standing beside his taxi in the middle of a \
street and giving instruction between the trucks that are parked on the street",
"person is standing on the table spraying a hose","person is crossing the busy street and watching a taxi speed off",\
"person is sitting on a hospital bed and he is afraid of", "person x is meanly looking at person y like he is about to throw a punch",
"person x is in uniform signaling for the life boat to lower and looking at person y in disgust",
"person sits in the lifeboat with his eyes closed looking ashamed and relieved",
"doctor examining person on a table",
"person x and person y embrace one another",
"at a fancy party pesron is trying to talk to the pretty woman in front of him"]

events_and_intents = []
for event in event_:
    set_seed(64)
    prompt = '''generate intent of action:
    person is facing the crowd while holding a microphone with intent to => make a public statement.
    pesron is sitting at the bar in a sexy dress and waiting for someone with intent to => flirt with a man.
    person is an officer that has handcuffed a citizen to himself with intent to => arrest the citizen.
    person is carrying her meal to her table with intent to => sit down to eat.
    {} with intent to => '''.format(event)
    print("EVENT: ",event)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    intents = []
    for i in range(10):
        
        generated_ids = model.generate(input_ids, do_sample=True, max_length=160)
        intent = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        #print("1    --->",intent)
        intent = intent[0].split(event)[1].replace("\n",".").split(".")[0].split("=>")
        
        if len(intent) > 1:
            intent = intent[1]
        else:
            continue
        intents.append(intent.lstrip())
        print("2    --->",intent.lstrip())
    events_and_intents.append((event, intents))
print(events_and_intents)

In [ ]:

event_ = ["a person plays a video game centered around ice age the movie","men are playing sports","girl with wings dances and sings the ole mcdonald song","person is looking down at a paper", "has hands in pockets", "person is taking a tiny spoon and scooping up a heap of caviar",
"person is in the background conversing with others in the room", \
"person is full of wounds and is standing beside his taxi in the middle of a \
street and giving instruction between the trucks that are parked on the street",
"person is standing on the table spraying a hose","person is crossing the busy street and watching a taxi speed off",\
"person is sitting on a hospital bed and he is afraid of", "person x is meanly looking at person y like he is about to throw a punch",
"person x is in uniform signaling for the life boat to lower and looking at person y in disgust",
"person sits in the lifeboat with his eyes closed looking ashamed and relieved",
"doctor examining person on a table",
"person x and person y embrace one another",
"at a fancy party pesron is trying to talk to the pretty woman in front of him"]

events_and_intents = []
for event in event_:
    set_seed(64)
    prompt = '''predict effect of action:
    person is facing the crowd while holding a microphone, as a result  => the public will engoy him speak.
    pesron is sitting at the bar in a sexy dress and waiting for someone, as a result => she will attract the attention of men.
    person is an officer that has handcuffed a citizen to himself, as a result =>  the citizen will be arrested and convicted.
    person is carrying her meal to her table, as a result => he will be full.
    {}, as a result => '''.format(event)
    print("EVENT: ",event)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    intents = []
    for i in range(10):
        
        generated_ids = model.generate(input_ids, do_sample=True, max_length=160)
        intent = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        #print("1    --->",intent)
        intent = intent[0].split(event)[1].replace("\n",".").split(".")[0].split("=>")
        #print(intent)
        if len(intent) > 1:
            intent = intent[1]
        else:
            continue
        intents.append(intent.lstrip())
        print("2    --->",intent.lstrip())
    events_and_intents.append((event, intents))
print(events_and_intents)

In [ ]:
set_seed(64)
prompt = '''
    Event: person is sitting at the bar in a sexy dress and waiting for someone. Similar events: man waiting for the train, lonely woman in pajamas eating chocolate.
    Event: person is facing the crowd while holding a microphone. Similar events: man making a presentation to men in suits, the singer performs on stage.
    Event: person is an officer that has handcuffed a citizen to himself. Similar events: female officer wearing a leather jacket and armed with a submachine gun, military man arresting a member of the public.
    Event: person is crossing the busy street and watching a taxi speed off. Similar events:
'''
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
intents = []
for i in range(5):
    
    generated_ids = model.generate(input_ids, do_sample=True, max_length=180)
    intent = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    #print("1    --->",intent)
    intent = intent[0].replace("\n",".").split("Event:")[-1]
    print("Debug:",intent)
    # if len(intent) > 1:
    #     intent = intent[1]
    # else:
    #     continue
    # intents.append(intent.lstrip())
    # print("2    --->",intent.lstrip())